In [1]:
import pandas as pd
import xgboost as xgb

In [5]:
import numpy as np
import sklearn as sk
#from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error

In [8]:
df = pd.read_csv('Water_Point_Data_Exchange_Complete_Dataset_binary.csv',encoding = 'latin-1')

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,6,8,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [81]:
pd.set_option('max_columns', 40)

In [82]:

df.head()

,Row ID,country_name,water_source,water_tech,status_id,status_0_yes,management,pay,installer,install_year,status,source,adm1,adm2,wpdx_id,report_date,country_id,activity_id,data_lnk,orig_lnk,photo_lnk,converted,created,updated,lat_deg,lon_deg,Location,Count,fecal_coliform_presence,fecal_coliform_value,subjective_quality
0,1,"Tanzania, United Republic of",Spring,Gravity Communal standpipe,no,1,nan,Never pay,RWE,1972.0,Status:Not functional|Quantity:Dry|Quality:Soft,SNV,Arusha,Longido,wpdx-00000001,01/05/2008 12:00:00 AM +0000,TZ,nan,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,nan,nan,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.83257,36.3090,"(-2.83257, 36.309)",1,NaN,NaN,NaN
1,2,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,0,nan,Never pay,World Vision,2005.0,Status:Functional|Breakdown Year:2007|Reason N...,SNV,Arusha,Longido,wpdx-00000002,01/01/2008 12:00:00 AM +0000,TZ,nan,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,nan,nan,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84781,36.5284,"(-2.84781, 36.5284)",1,NaN,NaN,NaN
2,3,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,0,nan,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,SNV,Arusha,Longido,wpdx-00000003,01/01/2008 12:00:00 AM +0000,TZ,nan,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,nan,nan,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84796,36.5288,"(-2.84796, 36.5288)",1,NaN,NaN,NaN
3,4,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,0,nan,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,SNV,Arusha,Longido,wpdx-00000004,01/01/2008 12:00:00 AM +0000,TZ,nan,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,nan,nan,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.85318,36.5276,"(-2.85318, 36.5276)",1,NaN,NaN,NaN
4,5,Nicaragua,nan,nan,yes,0,nan,nan,nan,2011.0,nan,El Porvenier,Matagalpa,Terrabona,wpdx-00000005,01/01/2011 12:00:00 AM +0000,NI,nan,http://www.dropbox.com/s/0f70mln5roxhy29/ProjT...,https://www.waterpointdata.org/dataset/resourc...,nan,"#data_lnk, #lon_deg, #lat_deg, #country_id",04/08/2016 08:35:49 AM +0000,04/08/2016 08:35:49 AM +0000,12.74070,-86.0121,"(12.7407, -86.0121)",1,NaN,NaN,NaN


In [36]:
df.dtypes

Row ID                        int64
country_name                 object
#water_source                object
#water_tech                  object
#status_id                   object
status_0_yes                  int64
#management                  object
#pay                         object
#installer                   object
#install_year               float64
#status                      object
#source                      object
#adm1                        object
#adm2                        object
#wpdx_id                     object
#report_date                 object
#country_id                  object
#activity_id                 object
#data_lnk                    object
#orig_lnk                    object
photo_lnk                    object
#converted                   object
#created                     object
#updated                     object
#lat_deg                    float64
#lon_deg                    float64
Location                     object
Count                       

In [87]:
def col_cln(df):
    #clean column names
    df.columns = [c.replace('#', '') for c in df.columns]
    #convery obj to str 
    #not working?
    for i in df.loc[:, df.dtypes == object]:
        df[i] = df[i].astype(str)
    return df

In [52]:
df = col_cln(df)
#df= cln_data_types(df)
print(df.columns)
df.dtypes

Index(['Row ID', 'country_name', 'water_source', 'water_tech', 'status_id',
       'status_0_yes', 'management', 'pay', 'installer', 'install_year',
       'status', 'source', 'adm1', 'adm2', 'wpdx_id', 'report_date',
       'country_id', 'activity_id', 'data_lnk', 'orig_lnk', 'photo_lnk',
       'converted', 'created', 'updated', 'lat_deg', 'lon_deg', 'Location',
       'Count', 'fecal_coliform_presence', 'fecal_coliform_value',
       'subjective_quality'],
      dtype='object')


Row ID                       int64
country_name                object
water_source                object
water_tech                  object
status_id                   object
status_0_yes                 int64
management                  object
pay                         object
installer                   object
install_year               float64
status                      object
source                      object
adm1                        object
adm2                        object
wpdx_id                     object
report_date                 object
country_id                  object
activity_id                 object
data_lnk                    object
orig_lnk                    object
photo_lnk                   object
converted                   object
created                     object
updated                     object
lat_deg                    float64
lon_deg                    float64
Location                    object
Count                        int64
fecal_coliform_prese

# fix the dates in install_year, report date so we can get a lifespan of the well

In [53]:
df.new_report_date = pd.to_datetime(df.report_date)

In [57]:
print(df.new_report_date[0:10])
print(df.report_date[0:10])
print(df.new_report_date[-10:])
print(df.report_date[-10:])

0   2008-01-05
1   2008-01-01
2   2008-01-01
3   2008-01-01
4   2011-01-01
5   2014-01-01
6   2012-01-01
7   2011-01-01
8   2011-01-01
9   2015-01-01
Name: report_date, dtype: datetime64[ns]
0    01/05/2008 12:00:00 AM +0000
1    01/01/2008 12:00:00 AM +0000
2    01/01/2008 12:00:00 AM +0000
3    01/01/2008 12:00:00 AM +0000
4    01/01/2011 12:00:00 AM +0000
5    01/01/2014 12:00:00 AM +0000
6    01/01/2012 12:00:00 AM +0000
7    01/01/2011 12:00:00 AM +0000
8    01/01/2011 12:00:00 AM +0000
9    01/01/2015 12:00:00 AM +0000
Name: report_date, dtype: object
397847   2012-03-23
397848   2009-11-01
397849   2014-02-13
397850   2009-12-01
397851   2014-12-06
397852   2015-06-11
397853   2016-04-19
397854   2009-12-01
397855   2016-09-05
397856   2016-04-03
Name: report_date, dtype: datetime64[ns]
397847    03/23/2012 12:00:00 AM +0000
397848    11/01/2009 12:00:00 AM +0000
397849    02/13/2014 12:00:00 AM +0000
397850    12/01/2009 12:00:00 AM +0000
397851    12/06/2014 12:00:00 AM +0000


In [75]:
#df.install_year = df.install_year.astype(float)
df.new_install_year = pd.to_datetime(df.install_year,format='%Y.0')
#df.dtypes

In [77]:
print(df.new_install_year[0:10])
print(df.install_year[0:10])

0   1972-01-01
1   2005-01-01
2   2005-01-01
3   2005-01-01
4   2011-01-01
5   2014-01-01
6   2012-01-01
7   2011-01-01
8   2011-01-01
9   2015-01-01
Name: install_year, dtype: datetime64[ns]
0    1972.0
1    2005.0
2    2005.0
3    2005.0
4    2011.0
5    2014.0
6    2012.0
7    2011.0
8    2011.0
9    2015.0
Name: install_year, dtype: float64


In [83]:
df.new_report_date[0:100] - df.new_install_year[0:100]

0    13153 days
1     1095 days
2     1095 days
3     1095 days
4        0 days
5        0 days
6        0 days
7        0 days
8        0 days
9        0 days
10       0 days
11       0 days
12       0 days
13       0 days
14       0 days
15       0 days
16       0 days
17       0 days
18       0 days
19       0 days
20       0 days
21       0 days
22       0 days
23       0 days
24       0 days
25       0 days
26       0 days
27       0 days
28       0 days
29       0 days
        ...    
70       0 days
71       0 days
72       0 days
73       0 days
74       0 days
75       0 days
76       0 days
77       0 days
78       0 days
79     734 days
80     734 days
81     734 days
82    2145 days
83     734 days
84     734 days
85     734 days
86    2925 days
87    2560 days
88    8769 days
89    1465 days
90    9347 days
91          NaT
92          NaT
93          NaT
94          NaT
95          NaT
96          NaT
97          NaT
98          NaT
99          NaT
dtype: timedelta64[ns]

In [86]:
df[91:100]

,Row ID,country_name,water_source,water_tech,status_id,status_0_yes,management,pay,installer,install_year,status,source,adm1,adm2,wpdx_id,report_date,country_id,activity_id,data_lnk,orig_lnk,photo_lnk,converted,created,updated,lat_deg,lon_deg,Location,Count,fecal_coliform_presence,fecal_coliform_value,subjective_quality
91,91,"Tanzania, United Republic of",Other,Gravity Other,unknown,1,nan,Not recorded,Unknown,NaN,Status:Not recorded|Quantity:Not recorded|Qual...,WA,Dodoma,Dodoma Rural,wpdx-00000091,08/04/2005 12:00:00 AM +0000,TZ,nan,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,nan,nan,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-6.49275,35.8337,"(-6.49275, 35.8337)",1,NaN,NaN,NaN
92,92,"Tanzania, United Republic of",Other,Gravity Other,unknown,1,nan,Not recorded,Unknown,NaN,Status:Not recorded|Quantity:Not recorded|Qual...,WA,Dodoma,Dodoma Rural,wpdx-00000092,08/04/2005 12:00:00 AM +0000,TZ,nan,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,nan,nan,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-6.49111,35.8324,"(-6.49111, 35.8324)",1,NaN,NaN,NaN
93,93,"Tanzania, United Republic of",Other,Gravity Other,unknown,1,nan,Not recorded,Unknown,NaN,Status:Not recorded|Quantity:Not recorded|Qual...,WA,Dodoma,Dodoma Rural,wpdx-00000093,08/04/2005 12:00:00 AM +0000,TZ,nan,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,nan,nan,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-6.44675,35.8653,"(-6.44675, 35.8653)",1,NaN,NaN,NaN
94,94,"Tanzania, United Republic of",Other,Gravity Other,unknown,1,nan,Not recorded,Unknown,NaN,Status:Not recorded|Quantity:Not recorded|Qual...,WA,Dodoma,Dodoma Rural,wpdx-00000094,08/04/2005 12:00:00 AM +0000,TZ,nan,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,nan,nan,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-6.43244,35.8650,"(-6.43244, 35.865)",1,NaN,NaN,NaN
95,95,"Tanzania, United Republic of",Other,Gravity Other,unknown,1,nan,Not recorded,Unknown,NaN,Status:Not recorded|Quantity:Not recorded|Qual...,WA,Dodoma,Dodoma Rural,wpdx-00000095,08/04/2005 12:00:00 AM +0000,TZ,nan,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,nan,nan,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-6.43238,35.8646,"(-6.43238, 35.8646)",1,NaN,NaN,NaN
96,96,"Tanzania, United Republic of",Other,Gravity Other,unknown,1,nan,Not recorded,Unknown,NaN,Status:Not recorded|Quantity:Not recorded|Qual...,WA,Dodoma,Dodoma Rural,wpdx-00000096,08/04/2005 12:00:00 AM +0000,TZ,nan,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,nan,nan,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-6.42992,35.8577,"(-6.42992, 35.8577)",1,NaN,NaN,NaN
97,97,"Tanzania, United Republic of",Other,Gravity Other,unknown,1,nan,Not recorded,Unknown,NaN,Status:Not recorded|Quantity:Not recorded|Qual...,WA,Dodoma,Dodoma Rural,wpdx-00000097,08/04/2005 12:00:00 AM +0000,TZ,nan,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,nan,nan,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-6.42717,35.8693,"(-6.42717, 35.8693)",1,NaN,NaN,NaN
98,98,"Tanzania, United Republic of",Machine-drilled borehole,Mono Communal standpipe,no,1,nan,Not recorded,Unknown,NaN,Status:Not functional|Quantity:Not recorded|Qu...,WA,Dodoma,Dodoma Rural,wpdx-00000098,08/04/2005 12:00:00 AM +0000,TZ,nan,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,nan,nan,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-6.42277,35.8674,"(-6.42277, 35.8674)",1,NaN,NaN,NaN
99,99,"Tanzania, United Republic of",Other,Gravity Other,unknown,1,nan,Not recorded,Unknown,NaN,Status:Not recorded|Quantity:Not recorded|Qual...,WA,Dodoma,Dodoma Rural,wpdx-00000099,08/04/2005 12:00:00 AM +0000,TZ,

# need to do something with missing install year

In [12]:
df.isnull().sum()/ len(df)

Row ID                      0.000000
#country_name               0.000000
#water_source               0.393556
#water_tech                 0.130265
#status_id                  0.000000
status_0_yes                0.000000
#management                 0.610350
#pay                        0.616513
#installer                  0.622103
#install_year               0.199356
#status                     0.280626
#source                     0.000000
#adm1                       0.144381
#adm2                       0.147998
#wpdx_id                    0.000000
#report_date                0.000000
#country_id                 0.000038
#activity_id                0.153595
#data_lnk                   0.224990
#orig_lnk                   0.598305
photo_lnk                   0.850527
#converted                  0.167259
#created                    0.000000
#updated                    0.000000
#lat_deg                    0.000000
#lon_deg                    0.000000
Location                    0.000000
C

In [13]:
df = df.rename(columns= ({'#country_name':'country_name'}))

In [14]:
df[df.country_name == 'Uganda'].isnull().sum()/len(df[df.country_name == 'Uganda'])

Row ID                      0.000000
country_name                0.000000
#water_source               0.873874
#water_tech                 0.107088
#status_id                  0.000000
status_0_yes                0.000000
#management                 0.117646
#pay                        0.646481
#installer                  0.981930
#install_year               0.155159
#status                     0.116167
#source                     0.000000
#adm1                       0.100642
#adm2                       0.100642
#wpdx_id                    0.000000
#report_date                0.000000
#country_id                 0.000000
#activity_id                0.012495
#data_lnk                   0.108770
#orig_lnk                   0.888835
photo_lnk                   0.974921
#converted                  0.097630
#created                    0.000000
#updated                    0.000000
#lat_deg                    0.000000
#lon_deg                    0.000000
Location                    0.000000
C

In [20]:
df2=df[['country_name','#water_source','#water_tech', '#installer','status_0_yes', '#management', '#install_year', '#pay', '#source']]

In [21]:
df2[df2.country_name == 'Uganda'].isnull().sum()/len(df2[df2.country_name == 'Uganda'])

country_name     0.000000
#water_source    0.873874
#water_tech      0.107088
#installer       0.981930
status_0_yes     0.000000
#management      0.117646
#install_year    0.155159
#pay             0.646481
#source          0.000000
dtype: float64

In [67]:
df2[df2.country_name == 'Afghanistan'].isnull().sum()/len(df2[df2.country_name == 'Afghanistan'])

country_name     0.000000
#water_source    0.497753
#water_tech      0.148942
#installer       0.095808
#status_id       0.000000
#management      1.000000
#install_year    0.084112
#pay             1.000000
#source          0.000000
dtype: float64

In [68]:
df2[df2.country_name == 'Sierra Leone'].isnull().sum()/len(df2[df2.country_name == 'Sierra Leone'])

country_name     0.000000
#water_source    0.000000
#water_tech      0.154783
#installer       0.089567
#status_id       0.000000
#management      0.510371
#install_year    0.101688
#pay             0.136137
#source          0.000000
dtype: float64

In [69]:
df2[df2.country_name == 'Swaziland'].isnull().sum()/len(df2[df2.country_name == 'Swaziland'])

country_name     0.00000
#water_source    0.00000
#water_tech      0.00000
#installer       1.00000
#status_id       0.00000
#management      0.64207
#install_year    0.00000
#pay             0.00000
#source          0.00000
dtype: float64

# Build baseline predictive model for Swaziland

In [27]:
swaz = df2[(df2.country_name == 'Swaziland')]

In [30]:
swaz.groupby(['#water_tech']).count()

,country_name,#water_source,#installer,status_0_yes,#management,#install_year,#pay,#source
#water_tech,,,,,,,,
BPT,349,349,0,349,81,349,349,349
Borehole,1467,1467,0,1467,840,1467,1467,1467
Canal,2,2,0,2,0,2,2,2
Control House,198,198,0,198,88,198,198,198
Dam,13,13,0,13,9,13,13,13
Hand Dug Well,3,3,0,3,0,3,3,3
Hand dug well,1,1,0,1,0,1,1,1
Pipeline,2,2,0,2,0,2,2,2
Protected Spring,171,171,0,171,66,171,171,171


In [31]:
swaz.groupby("country_name").apply(lambda x: x.notnull().mean())

,country_name,#water_source,#water_tech,#installer,status_0_yes,#management,#install_year,#pay,#source
country_name,,,,,,,,,
Swaziland,1.0,1.0,1.0,0.0,1.0,0.35793,1.0,1.0,1.0


In [23]:
#df2[(df2.country_name == 'Uganda')| (df2.country_name == 'Afghanistan')|(df2.country_name == 'Swaziland')]

In [88]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedKFold
import xgboost as xgb

class CustomModel(object):

    def fit(self, X, Y, extras=None):
        '''This is where you would fit your model to the data
        Parameters
        ----------
        X : pandas.DataFrame
            Contains your data - you can think of it as simply loaded from
            pandas.read_csv, but any transformed or derived features you
            have included come along
        Y : pandas.Series
            Has as many elements as X has rows - these are the predictions
            that go along with the data in X
        extras : dict
            Contains additional information that can be used for model fitting,
            for example, case_weights
        Returns
        -------
        self : CustomModel
            We utilize operator chaining and need to be able to run
            ``self.fit().predict()`` or similar.
        '''
        X_copy = X.copy()
        Y_copy = Y.copy()

        # Identify which columns need to be label encoded (everything except numeric)
        self.columns_label_encoded = {}
        for col in X_copy:
            if X_copy[col].dtype.name == 'object':
                self.columns_label_encoded[col] = LabelEncoder()

        # Do stratified split to get 10% internal valid partition for tuning
        skf = StratifiedKFold(y=Y, n_folds=10, shuffle=True)
        for train_indices, valid_indices in skf:
            break
        train_X = X_copy.iloc[train_indices]
        train_Y = Y_copy[train_indices]
        valid_X = X_copy.iloc[valid_indices]
        valid_Y = Y_copy[valid_indices]

        # Label encode all non numeric features
        for col, le in self.columns_label_encoded.items():
            le.fit(list(train_X[col].fillna('___MISSING___').apply(unicode).drop_duplicates().values) +
                   ['___MISSING___'])
            train_X[col] = le.transform(train_X[col].fillna('___MISSING___').apply(unicode))

        for col, le in self.columns_label_encoded.items():
            valid_X[col] = le.transform(self.fix_for_new_vocabulary(valid_X[col]))

        self.train_columns = X.columns

        dtrain = xgb.DMatrix(data=train_X[self.train_columns].values, label=train_Y, missing=np.nan)
        dvalid = xgb.DMatrix(data=valid_X[self.train_columns].values, label=valid_Y, missing=np.nan)
        param = {'silent': 1,
                 'objective':'binary:logistic',
                 'eval_metric': 'logloss'}
        watchlist  = [(dtrain, 'train'), (dvalid, 'eval')]
        num_round = 5000

        self.bst_cv = xgb.train(params=param,
                                dtrain=dtrain,
                                num_boost_round=num_round,
                                evals=watchlist,
                                early_stopping_rounds=50)

        # Build for full training data
        for col, le in self.columns_label_encoded.items():
            le.fit(list(X[col].fillna('___MISSING___').apply(unicode).drop_duplicates().values) +
                   ['___MISSING___'])
            X_copy[col] = le.transform(X_copy[col].fillna('___MISSING___').apply(unicode))

        dtrain_full = xgb.DMatrix(data=X_copy[self.train_columns].values, label=Y, missing=np.nan)
        self.bst = xgb.train(params=param,
                             dtrain=dtrain_full,
                             num_boost_round=self.bst_cv.best_iteration)

        return self

    def fix_for_new_vocabulary(self, series):
        s = series.copy()
        s = s.apply(unicode)
        new_words = set(s) - set(self.columns_label_encoded[s.name].classes_)
        if len(new_words) > 0:
            s = s.replace(to_replace=list(new_words), value='___MISSING___').fillna('___MISSING___')
        return s


    def predict(self, X):
        '''This is the prediction method that you would call
        The output is a numpy ndarray, having a single column and as many
        rows as X
        Parameters
        ----------
        X : pandas.DataFrame
            The data on which to make a prediction using your newly fit model

        Returns
        -------
        Y : numpy.ndarray
            With a single column, and the same number of rows as ``X``
        '''
        X_copy = X.copy()
        # Label encode all non numeric features
        for col, le in self.columns_label_encoded.items():
            X_copy[col] = le.transform(self.fix_for_new_vocabulary(X_copy[col]))

        dtest = xgb.DMatrix(data=X_copy[self.train_columns].values, missing=np.nan)
        result = self.bst.predict(dtest)
        return result.reshape(-1, 1)


In [33]:
swaz.columns

Index(['country_name', '#water_source', '#water_tech', '#installer',
       'status_0_yes', '#management', '#install_year', '#pay', '#source'],
      dtype='object')

In [34]:
cols = ['#water_source','#water_tech', '#installer',
       'status_0_yes', '#management', '#install_year', '#pay', '#source']

In [35]:
swaz[cols]

,#water_source,#water_tech,#installer,status_0_yes,#management,#install_year,#pay,#source
301736,Borehole fitted with submersible pump,Standpipe,NaN,1,Community Management,1982.0,15,Swaziland Department of Water Affairs
301737,Borehole fitted with submersible pump,Standpipe,NaN,1,Community Management,1982.0,15,Swaziland Department of Water Affairs
301738,Borehole fitted with submersible pump,Standpipe,NaN,1,Community Management,1982.0,15,Swaziland Department of Water Affairs
301739,Borehole fitted with submersible pump,Standpipe,NaN,1,Community Management,1982.0,15,Swaziland Department of Water Affairs
301740,Borehole fitted with submersible pump,Standpipe,NaN,1,Community Management,1982.0,15,Swaziland Department of Water Affairs
301741,Borehole fitted with submersible pump,Standpipe,NaN,1,Community Management,1982.0,15,Swaziland Department of Water Affairs
301742,Borehole fitted with submersible pump,Standpipe,NaN,1,Community Management,1982.0,15,Swaziland Department of Water Affairs
301743,Borehole fitted with submersible pump,Standpipe,NaN,0,Community Management,1980.0,20,Swaziland Department of Water Affairs
301744,Borehole fitted with submersible pump,Standpipe,NaN,0,Community Management,1984.0,20,Swaziland Department of Water Affairs
301745,gravity river/stream,Standpipe,NaN,0,Community Management,2015.0,20,Swaziland Department of Water Affairs
